In [0]:
import torch
import torchvision
from torchvision import transforms as transforms
import torch.nn as nn

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
# Hyper parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size= 100
learning_rate = 0.001

In [0]:
# Extract
train_dataset = torchvision.datasets.MNIST(root='../data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = torchvision.datasets.MNIST(root='../data', train=False, download=True, transform=transforms.ToTensor())

In [0]:
# Transform
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [0]:
class Network(nn.Module):
    
        def __init__(self, input_size, hidden_size, num_classes):
            super(Network, self).__init__()
            self.fc1 = nn.Linear(input_size, hidden_size)
            self.relu = nn.ReLU()
            self.fc2 = nn.Linear(hidden_size, 200)
            self.fc3 = nn.Linear(200, num_classes)

        def forward(self, x):
            out = self.fc1(x)
            out = self.relu(out)
            out = self.fc2(out)
            out = self.relu(out)
            out = self.fc3(out)
            return out
        
        

In [0]:
network = Network(input_size, hidden_size, num_classes).to(device)

In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)

In [0]:
# Train
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # load
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # forward
        outputs = network(images)
        loss = criterion(outputs, labels)

        # Back prop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        

In [0]:
#Test
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = network(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('{}%'.format(100* correct / total))

torch.save(network.state_dict(), 'model.ckpt')

97.95%
